# Pandas `groupby()`

`groupby()` works through three steps:  
**Split** the data into groups → **Apply** an operation → **Combine** the results.

Operations include:  
- **Aggregation** (mean, sum, count)  
- **Transformation** (group-wise z-score, scaling)  
- **Filtration** (remove groups based on a rule)

This pattern makes `groupby()` essential for multi-stock analysis, sector studies, time-based analysis and portfolio grouping.

In [1]:
# Creating a DataFrame

import pandas as pd

my_portfolio = {
    'Sector': [
        'IT', 'FMCG', 'Banking', 'Pharma', 'Auto',
        'FMCG', 'Oil & Gas', 'IT', 'Banking', 'Real Estate'],

    'Company': [
        'Infosys', 'Hindustan Unilever', 'HDFC Bank', 'Sun Pharma', 'Tata Motors',
        'Britannia', 'Oil India', 'Tata Elxsi', 'ICICI Bank', 'Godrej Properties'],

    'MarketCap': [
        'Large Cap', 'Large Cap', 'Large Cap', 'Large Cap', 'Mid Cap',
        'Large Cap', 'Mid Cap', 'Small Cap', 'Large Cap', 'Small Cap'],

    'Share Price': [
        1600, 2450, 1650, 1450, 980,
        5250, 510, 8800, 1120, 1900 ],

    'Amount Invested': [
        40000, 25000, 30000, 20000, 15000,
        35000, 18000, 25000, 20000, 12000] }

ttp = pd.DataFrame(my_portfolio)
ttp


,Sector,Company,MarketCap,Share Price,Amount Invested
0,IT,Infosys,Large Cap,1600,40000
1,FMCG,Hindustan Unilever,Large Cap,2450,25000
2,Banking,HDFC Bank,Large Cap,1650,30000
3,Pharma,Sun Pharma,Large Cap,1450,20000
4,Auto,Tata Motors,Mid Cap,980,15000
5,FMCG,Britannia,Large Cap,5250,35000
6,Oil & Gas,Oil India,Mid Cap,510,18000
7,IT,Tata Elxsi,Small Cap,8800,25000
8,Banking,ICICI Bank,Large Cap,1120,20000
9,Real Estate,Godrej Properties,Small Cap,1900,12000


### View groups

In [2]:
#Output is keys='MarketCap' and values= the index labels/row numbers
ttp.groupby('MarketCap').groups

{'Large Cap': [0, 1, 2, 3, 5, 8], 'Mid Cap': [4, 6], 'Small Cap': [7, 9]}

In [3]:
ttp.groupby('Sector').groups

{'Auto': [4], 'Banking': [2, 8], 'FMCG': [1, 5], 'IT': [0, 7], 'Oil & Gas': [6], 'Pharma': [3], 'Real Estate': [9]}

In [4]:
# Groupby with multiple columns

ttp.groupby(['MarketCap', 'Sector']).groups

{('Large Cap', 'Banking'): [2, 8], ('Large Cap', 'FMCG'): [1, 5], ('Large Cap', 'IT'): [0], ('Large Cap', 'Pharma'): [3], ('Mid Cap', 'Auto'): [4], ('Mid Cap', 'Oil & Gas'): [6], ('Small Cap', 'IT'): [7], ('Small Cap', 'Real Estate'): [9]}

### Iterating through groups (a better way to see)

In [5]:
# A better way to visualise
# Create separate object(mini-dataframes container) for grouping each sector
grouped = ttp.groupby('Sector')

# The loop goes through each (key, subset) pair in the grouped object.
# name → the group name (e.g., 'IT', 'Banking', 'FMCG')
# group → the mini-DataFrame containing only the rows belonging to that sector.
# This prints the sector name (the group key).

for name, group in grouped:
    print(name)
    print(group)

Auto
  Sector      Company MarketCap  Share Price  Amount Invested
4   Auto  Tata Motors   Mid Cap          980            15000
Banking
    Sector     Company  MarketCap  Share Price  Amount Invested
2  Banking   HDFC Bank  Large Cap         1650            30000
8  Banking  ICICI Bank  Large Cap         1120            20000
FMCG
  Sector             Company  MarketCap  Share Price  Amount Invested
1   FMCG  Hindustan Unilever  Large Cap         2450            25000
5   FMCG           Britannia  Large Cap         5250            35000
IT
  Sector     Company  MarketCap  Share Price  Amount Invested
0     IT     Infosys  Large Cap         1600            40000
7     IT  Tata Elxsi  Small Cap         8800            25000
Oil & Gas
      Sector    Company MarketCap  Share Price  Amount Invested
6  Oil & Gas  Oil India   Mid Cap          510            18000
Pharma
   Sector     Company  MarketCap  Share Price  Amount Invested
3  Pharma  Sun Pharma  Large Cap         1450            20

In [6]:
# Lets try with MarketCap now

grouped = ttp.groupby('MarketCap')

for name, group in grouped:  # We will learn 'for' loop in further sections. It is usually used for iterations
    print(name)
    print(group)

Large Cap
    Sector             Company  MarketCap  Share Price  Amount Invested
0       IT             Infosys  Large Cap         1600            40000
1     FMCG  Hindustan Unilever  Large Cap         2450            25000
2  Banking           HDFC Bank  Large Cap         1650            30000
3   Pharma          Sun Pharma  Large Cap         1450            20000
5     FMCG           Britannia  Large Cap         5250            35000
8  Banking          ICICI Bank  Large Cap         1120            20000
Mid Cap
      Sector      Company MarketCap  Share Price  Amount Invested
4       Auto  Tata Motors   Mid Cap          980            15000
6  Oil & Gas    Oil India   Mid Cap          510            18000
Small Cap
        Sector            Company  MarketCap  Share Price  Amount Invested
7           IT         Tata Elxsi  Small Cap         8800            25000
9  Real Estate  Godrej Properties  Small Cap         1900            12000


### Select a group or to get one group

In [7]:
grouped = ttp.groupby('MarketCap')

print(grouped.get_group('Mid Cap'))
print("---------------------------------------------------------------------------------")
print(grouped.get_group('Large Cap'))

      Sector      Company MarketCap  Share Price  Amount Invested
4       Auto  Tata Motors   Mid Cap          980            15000
6  Oil & Gas    Oil India   Mid Cap          510            18000
---------------------------------------------------------------------------------
    Sector             Company  MarketCap  Share Price  Amount Invested
0       IT             Infosys  Large Cap         1600            40000
1     FMCG  Hindustan Unilever  Large Cap         2450            25000
2  Banking           HDFC Bank  Large Cap         1650            30000
3   Pharma          Sun Pharma  Large Cap         1450            20000
5     FMCG           Britannia  Large Cap         5250            35000
8  Banking          ICICI Bank  Large Cap         1120            20000


### Aggregations

In [8]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")

grouped = ttp.groupby('MarketCap')

print(grouped['Amount Invested'].agg(np.mean))

MarketCap
Large Cap    28333.333333
Mid Cap      16500.000000
Small Cap    18500.000000
Name: Amount Invested, dtype: float64


This means that on an average, we have invested Rs. 28333 per script in Large Cap, Rs. 16500 per script in Mid Cap and Rs. 18500 per script in Small Cap

In [10]:
# Applying multiple aggregation functions at once

grouped = ttp.groupby('MarketCap')

print(grouped['Amount Invested'].agg([np.sum, np.mean]))

              sum          mean
MarketCap                      
Large Cap  170000  28333.333333
Mid Cap     33000  16500.000000
Small Cap   37000  18500.000000


In [11]:
print(np.round(grouped['Amount Invested'].agg([np.sum, np.mean]),2))

              sum      mean
MarketCap                  
Large Cap  170000  28333.33
Mid Cap     33000  16500.00
Small Cap   37000  18500.00


In [12]:
grouped = ttp.groupby('MarketCap')

### Transformations

In [14]:
#This function takes a Series x and standardizes it
def z_score(x): return (x - x.mean()) / x.std()

print(grouped[['Share Price', 'Amount Invested']].transform(z_score))

   Share Price  Amount Invested
0    -0.426381         1.428869
1     0.128349        -0.408248
2    -0.393750         0.204124
3    -0.524275        -1.020621
4     0.707107        -0.707107
5     1.955698         0.816497
6    -0.707107         0.707107
7     0.707107         0.707107
8    -0.739641        -1.020621
9    -0.707107        -0.707107


**Applies the z_score function to each numeric column**

### Filteration

In [16]:
print(ttp.groupby('MarketCap').filter(lambda x: len(x) <= 3))

        Sector            Company  MarketCap  Share Price  Amount Invested
4         Auto        Tata Motors    Mid Cap          980            15000
6    Oil & Gas          Oil India    Mid Cap          510            18000
7           IT         Tata Elxsi  Small Cap         8800            25000
9  Real Estate  Godrej Properties  Small Cap         1900            12000


**It will filter out the Groups that have less than 3 companies in that particular group.**

### Merging/Joining 

In [17]:
import pandas as pd

left_df = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'Company': ['HCL Tech', 'Axis Bank', 'Nestle India', 'Eicher Motors', 'Cipla'],
    'Sector': ['IT', 'Banking', 'FMCG', 'Auto', 'Pharma']
})

right_df = pd.DataFrame(
    {'id': [1, 2, 3, 4, 5],
     'Company': ['Power Grid', 'Tech Mahindra', 'Dr Reddy\'s Labs', 'IndusInd Bank', 'Hero MotoCorp'],
     'Sector': ['Power', 'IT', 'Pharma', 'Banking', 'Auto']})

In [18]:
left_df

,id,Company,Sector
0,1,HCL Tech,IT
1,2,Axis Bank,Banking
2,3,Nestle India,FMCG
3,4,Eicher Motors,Auto
4,5,Cipla,Pharma


In [19]:
right_df

,id,Company,Sector
0,1,Power Grid,Power
1,2,Tech Mahindra,IT
2,3,Dr Reddy's Labs,Pharma
3,4,IndusInd Bank,Banking
4,5,Hero MotoCorp,Auto


### Syntax  
`pd.merge(left_df, right_df, on='id')`

### Explanation
- Performs an **inner join** (default join type)
- Joins rows **only where `id` matches** in both dataframes
- Keeps **all columns** from both dataframes
- Since both dataframes have columns with the same names (`Company`, `Sector`), pandas automatically adds suffixes:
  - `Company_x` → from `left_df`
  - `Company_y` → from `right_df`
  - `Sector_x` → from `left_df`
  - `Sector_y` → from `right_df`



In [20]:
# Merge 2 DFs on a key

print(pd.merge(left_df, right_df, on='id'))

   id      Company_x Sector_x        Company_y Sector_y
0   1       HCL Tech       IT       Power Grid    Power
1   2      Axis Bank  Banking    Tech Mahindra       IT
2   3   Nestle India     FMCG  Dr Reddy's Labs   Pharma
3   4  Eicher Motors     Auto    IndusInd Bank  Banking
4   5          Cipla   Pharma    Hero MotoCorp     Auto


### Merge on Sector

- Merges rows where **Sector values match** in both dataframes.  
- Duplicate column names get suffixes `_x` and `_y` to indicate their source.

In [21]:
print(pd.merge(left_df, right_df, on='Sector'))

   id_x      Company_x   Sector  id_y        Company_y
0     1       HCL Tech       IT     2    Tech Mahindra
1     2      Axis Bank  Banking     4    IndusInd Bank
2     4  Eicher Motors     Auto     5    Hero MotoCorp
3     5          Cipla   Pharma     3  Dr Reddy's Labs


In [22]:
# Merge 2 DFs on multiple keys

print(pd.merge(left_df, right_df, on=['Sector', 'Company']))

Empty DataFrame
Columns: [id_x, Company, Sector, id_y]
Index: []


**Output above is empty as there are no rows where BOTH sector and company match.**

### Left Join on Sector

➡ Keeps all rows from `left_df`, and matches data from `right_df` where `Sector` names are the same.  
➡ If no match is found, columns from `right_df` appear as **NaN**.  

#### Explanation
- Every row from `left_df` is preserved.  
- Example: For `FMCG`, there is no matching Sector in `right_df`, so merged values become **NaN**.

In [23]:
# Merge using 'how' argument

# Left join

print(pd.merge(left_df, right_df, on='Sector', how='left'))

   id_x      Company_x   Sector  id_y        Company_y
0     1       HCL Tech       IT   2.0    Tech Mahindra
1     2      Axis Bank  Banking   4.0    IndusInd Bank
2     3   Nestle India     FMCG   NaN              NaN
3     4  Eicher Motors     Auto   5.0    Hero MotoCorp
4     5          Cipla   Pharma   3.0  Dr Reddy's Labs


### Right Join on Sector

➡ Keeps all rows from `right_df`, and matches data from `left_df` where `Sector` values are the same.  
➡ If no match is found, columns from `left_df` appear as **NaN**.  

#### Explanation
- Every row from `right_df` is preserved.  
- Example: If a Sector exists only in `right_df`, its matching values from `left_df` become **NaN**.

In [24]:
# Right join
print(pd.merge(left_df, right_df, on='Sector', how='right'))

   id_x      Company_x   Sector  id_y        Company_y
0   NaN            NaN    Power     1       Power Grid
1   1.0       HCL Tech       IT     2    Tech Mahindra
2   5.0          Cipla   Pharma     3  Dr Reddy's Labs
3   2.0      Axis Bank  Banking     4    IndusInd Bank
4   4.0  Eicher Motors     Auto     5    Hero MotoCorp


### Outer Join on Sector

➡ Keeps **all rows from both DataFrames**.  
➡ If a value exists only in one DataFrame, the other side is filled with **NaN**.  

#### Explanation
- Includes all unique Sectors from both tables: IT, Banking, FMCG, Auto, Pharma, Power.  
- Where a Sector matches → data is merged.  
- Where no match exists → the unmatched side becomes **NaN**.

In [26]:
# Outer join

print(pd.merge(left_df, right_df, on='Sector', how='outer'))

   id_x      Company_x   Sector  id_y        Company_y
0   4.0  Eicher Motors     Auto   5.0    Hero MotoCorp
1   2.0      Axis Bank  Banking   4.0    IndusInd Bank
2   3.0   Nestle India     FMCG   NaN              NaN
3   1.0       HCL Tech       IT   2.0    Tech Mahindra
4   5.0          Cipla   Pharma   3.0  Dr Reddy's Labs
5   NaN            NaN    Power   1.0       Power Grid


### Inner Join on Sector

➡ Keeps only rows where **Sector** matches in both DataFrames; all non-matching sectors are dropped.  

#### Explanation
- Only sectors common to both tables remain: **IT, Banking, Auto, Pharma**.  
- Sectors like **FMCG** (left only) and **Power** (right only) are removed.

In [27]:
# Inner join

print(pd.merge(left_df, right_df, on='Sector', how='inner'))

   id_x      Company_x   Sector  id_y        Company_y
0     1       HCL Tech       IT     2    Tech Mahindra
1     2      Axis Bank  Banking     4    IndusInd Bank
2     4  Eicher Motors     Auto     5    Hero MotoCorp
3     5          Cipla   Pharma     3  Dr Reddy's Labs


| Join Type | Rows Kept From    | Non-Matching Rows Show As | Common Sectors     |
| --------- | ----------------- | ------------------------- | ------------------ |
| `left`    | left_df           | NaN on right side         | All from left      |
| `right`   | right_df          | NaN on left side          | All from right     |
| `outer`   | both              | NaN where missing         | All from both      |
| `inner`   | intersection only | none                      | Only matching ones |


### Concatenation (Row-wise)

➡ Stacks `left_df` and `right_df` **vertically**, one below the other.  
➡ Does **not** match on any column; it simply appends rows and keeps all original columns.

#### Explanation
- Resulting DataFrame has **10 rows** (5 from each).  
- Columns from both DataFrames appear together, and non-overlapping data becomes **NaN** where missing, this only happens when the two DataFrames do NOT have the same set of columns.

In [28]:
print(left_df)

   id        Company   Sector
0   1       HCL Tech       IT
1   2      Axis Bank  Banking
2   3   Nestle India     FMCG
3   4  Eicher Motors     Auto
4   5          Cipla   Pharma


In [29]:
print(right_df)

   id          Company   Sector
0   1       Power Grid    Power
1   2    Tech Mahindra       IT
2   3  Dr Reddy's Labs   Pharma
3   4    IndusInd Bank  Banking
4   5    Hero MotoCorp     Auto


In [30]:
print(pd.concat([left_df, right_df]))

   id          Company   Sector
0   1         HCL Tech       IT
1   2        Axis Bank  Banking
2   3     Nestle India     FMCG
3   4    Eicher Motors     Auto
4   5            Cipla   Pharma
0   1       Power Grid    Power
1   2    Tech Mahindra       IT
2   3  Dr Reddy's Labs   Pharma
3   4    IndusInd Bank  Banking
4   5    Hero MotoCorp     Auto


### Concatenation with Keys (Hierarchical Index)

➡ Stacks `right_df` below `left_df`, but assigns a **multi-level index** using the keys `x` and `y`.  
➡ Using `ignore_index=True` removes the hierarchical index and gives a simple continuous index.

#### Explanation
- With keys: Index shows **x** for rows from `left_df` and **y** for rows from `right_df`.  
- With `ignore_index=True`: Hierarchical index is dropped and rows are renumbered from 0 onward.


In [31]:
print(pd.concat([left_df, right_df], keys=['x', 'y']))

     id          Company   Sector
x 0   1         HCL Tech       IT
  1   2        Axis Bank  Banking
  2   3     Nestle India     FMCG
  3   4    Eicher Motors     Auto
  4   5            Cipla   Pharma
y 0   1       Power Grid    Power
  1   2    Tech Mahindra       IT
  2   3  Dr Reddy's Labs   Pharma
  3   4    IndusInd Bank  Banking
  4   5    Hero MotoCorp     Auto


In [33]:
print(pd.concat([left_df, right_df], keys=['x', 'y'], ignore_index=True))

   id          Company   Sector
0   1         HCL Tech       IT
1   2        Axis Bank  Banking
2   3     Nestle India     FMCG
3   4    Eicher Motors     Auto
4   5            Cipla   Pharma
5   1       Power Grid    Power
6   2    Tech Mahindra       IT
7   3  Dr Reddy's Labs   Pharma
8   4    IndusInd Bank  Banking
9   5    Hero MotoCorp     Auto


### Concatenation Column-wise (axis=1)

➡ Joins the two DataFrames **side by side** instead of stacking vertically.  

#### Explanation
- Columns from both DataFrames are combined.  
- Row 1 of `left_df` aligns with Row 1 of `right_df`, Row 2 with Row 2, etc.  
- Duplicate column names receive suffixes `_x` and `_y` to avoid conflicts.  
- Works like placing two Excel tables next to each other.
- It simply places the two DataFrames side-by-side, matching rows by index,Column names do NOT matter here

In [35]:
print(pd.concat([left_df, right_df], axis=1))

   id        Company   Sector  id          Company   Sector
0   1       HCL Tech       IT   1       Power Grid    Power
1   2      Axis Bank  Banking   2    Tech Mahindra       IT
2   3   Nestle India     FMCG   3  Dr Reddy's Labs   Pharma
3   4  Eicher Motors     Auto   4    IndusInd Bank  Banking
4   5          Cipla   Pharma   5    Hero MotoCorp     Auto


### Concatenating Using append (Vertical)

➡ Stacks `right_df` below `left_df` in a **vertical concatenation**.

#### Explanation
- Adds all rows from both DataFrames sequentially.  
- Keeps the original index values, so indices like **0–4 repeat**.  
- Columns align automatically because both DataFrames have the same structure.
- NaN appear when the two DataFrames have different column names

### Concatenating Multiple DataFrames

➡ Stacks four DataFrames sequentially: first `left_df`, then `right_df`, then `left_df` again, and finally `right_df`.

#### Explanation
- Concatenates all DataFrames from the list `[left_df, right_df, left_df, right_df]`.  
- Total rows = **5 + 5 + 5 + 5 = 20**.  
- Same columns across all, so the index values repeat.  
- You can reset the index afterwards if needed.
- Think of it like copying and pasting the same two tables one below the other, twice

In [36]:
print(pd.concat([left_df, right_df, left_df, right_df]))

   id          Company   Sector
0   1         HCL Tech       IT
1   2        Axis Bank  Banking
2   3     Nestle India     FMCG
3   4    Eicher Motors     Auto
4   5            Cipla   Pharma
0   1       Power Grid    Power
1   2    Tech Mahindra       IT
2   3  Dr Reddy's Labs   Pharma
3   4    IndusInd Bank  Banking
4   5    Hero MotoCorp     Auto
0   1         HCL Tech       IT
1   2        Axis Bank  Banking
2   3     Nestle India     FMCG
3   4    Eicher Motors     Auto
4   5            Cipla   Pharma
0   1       Power Grid    Power
1   2    Tech Mahindra       IT
2   3  Dr Reddy's Labs   Pharma
3   4    IndusInd Bank  Banking
4   5    Hero MotoCorp     Auto
